In [ ]:
!pip install "pydantic-ai[all]" litellm opentelemetry-sdk opentelemetry-exporter-otlp python-dotenv pip-system-certs

In [10]:
!pip install --upgrade certifi

In [1]:
# config.py
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=".env",              # load from .env
        env_file_encoding="utf-8",
    )

    openrouter_api_key: str
    openrouter_base_url: str = "https://openrouter.ai/api/v1"

settings = Settings()

In [ ]:
import logging
import httpx
import asyncio

from litellm import completion
from openai import RateLimitError
from opentelemetry.sdk._logs import LoggerProvider
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor, ConsoleLogExporter

from pydantic_ai import Agent, InstrumentationSettings, ModelSettings
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.litellm import LiteLLMProvider
from pydantic_ai.models.openrouter import OpenRouterModelSettings

from pydantic_ai.exceptions import ModelHTTPError
from pydantic_ai.models.fallback import FallbackModel

# ---------- Basic logging ----------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
)
logger = logging.getLogger("openrouter_agent")

# ---------- OpenTelemetry setup ----------
resource = Resource.create({"service.name": "openrouter-agent"})

tracer_provider = TracerProvider(resource=resource)
tracer_provider.add_span_processor(
    BatchSpanProcessor(ConsoleSpanExporter())
)

logger_provider = LoggerProvider(resource=resource)
logger_provider.add_log_record_processor(
    BatchLogRecordProcessor(ConsoleLogExporter())
)

instrumentation_settings = InstrumentationSettings(
    tracer_provider=tracer_provider,
    logger_provider=logger_provider
)

# ---------- PydanticAI agent (Functional approach) ----------
from typing import Callable, Awaitable
from functools import partial

# Pure function to create provider configuration
def create_provider_config(settings:Settings, http_client: httpx.AsyncClient) -> dict:
    """Pure function returning provider configuration."""
    return {
        "api_base": settings.openrouter_base_url,
        "api_key": settings.openrouter_api_key,
        "http_client": http_client, 
    }

# Pure function to create model settings
def create_model_settings() -> ModelSettings:
    return ModelSettings(
        max_tokens=500,
    )

# Functional composition: provider -> model -> agent
def build_provider(config: dict) -> LiteLLMProvider:
    """Build provider from configuration."""
    return LiteLLMProvider(
        api_key=config.get("api_key",  None),
        http_client=config.get("http_client", None),
        api_base=config.get("api_base", None),
    )

def build_model(provider: LiteLLMProvider) -> FallbackModel:
    """Build model with provider."""
    openrouter_models=[
        "google/gemini-2.0-flash-exp:free",
        "nex-agi/deepseek-v3.1-nex-n1:free",   # Fallback 1
        "mistralai/devstral-2512:free",        # Fallback 2
    ]

    # Create model instances for the fallback models
    model_instances: list[OpenAIChatModel] = []
    for model_name in openrouter_models:
        model = OpenAIChatModel(
            model_name=model_name,
            provider=provider,
            settings=create_model_settings(),
        )
        model_instances.append(model)

    # Initialize FallbackModel with a default and fallbacks
    # The first model in the list is used as the default_model    
    fallbacks = FallbackModel(
        model_instances[0],
        *model_instances[1:]  # Pass as list, not unpacked
    )
    
    return fallbacks

def build_agent(model: FallbackModel, system_prompt: str) -> Agent:
    """Build agent with model and instrumentation."""
    return Agent(
        model,
        system_prompt=system_prompt,
        instrument=instrumentation_settings
    )

# Fluent builder pattern for agent creation
def create_agent(http_client: httpx.AsyncClient) -> Agent:
    """
    Fluent pipeline for agent creation.
    Settings -> Provider -> Model -> Agent
    """
    SYSTEM_PROMPT = "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."
    
    settings = Settings()
    config = create_provider_config(settings, http_client)
    provider = build_provider(config)
    model = build_model(provider)
    return build_agent(model, SYSTEM_PROMPT)

def with_logging(func: Callable[..., Awaitable]) -> Callable[..., Awaitable]:
    """Decorator for adding logging to async functions."""
    async def wrapper(*args, **kwargs):
        logger.info(f"Starting {func.__name__}...")
        result = await func(*args, **kwargs)
        logger.info(f"{func.__name__} finished.")
        return result
    return wrapper

# Pure async function for running agent query
@with_logging
async def run_agent_query(agent: Agent, question: str, retries: int = 1) -> str:
    """Pure async function to run agent query and return output."""
    for i in range(retries):
        try:
            result = await agent.run(question)
            return result.output
        except ModelHTTPError as e:
            if e.status_code == 429:
                logger.warning(f"OpenRouter 429 rate limit: {e.body}. Retrying...")
                if i < retries - 1:
                    await asyncio.sleep((i + 1) * 5)
                    continue
            raise  # Re-raise non-429 or final retry       
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            raise
            
# Compose the entire workflow
async def execute_agent_workflow(http_client: httpx.AsyncClient, question: str) -> str:
    """
    Functional workflow: create agent -> run query -> return result.
    This is a pure functional pipeline.
    """
    agent = create_agent(http_client)
    return await run_agent_query(agent, question)

# Main entry point using context manager and asyncio.run()
async def main() -> None:
    """Main function with proper resource management."""
    question = input("What do you need help with?")
    
    # Context manager for automatic resource cleanup (fluent Python)
    async with httpx.AsyncClient(verify=False, timeout=60.0) as http_client:
        answer = await execute_agent_workflow(http_client, question)
        print(f"Agent answer:\n{answer}")

# Pythonic entry point
if __name__ == "__main__":
    await main()


C:\Users\cpu\AppData\Local\Temp\ipykernel_34644\2588355798.py:38: DeprecationWarning: Use ConsoleLogRecordExporter. Since logs are not stable yet this WILL be removed in future releases.
  BatchLogRecordProcessor(ConsoleLogExporter())
2025-12-29 23:29:41,788 [INFO] openrouter_agent - Starting run_agent_query...
2025-12-29 23:29:42,195 [INFO] openai._base_client - Retrying request to /chat/completions in 0.441977 seconds
2025-12-29 23:29:42,786 [INFO] openai._base_client - Retrying request to /chat/completions in 0.902715 seconds
2025-12-29 23:29:43,927 [INFO] openai._base_client - Retrying request to /chat/completions in 0.406654 seconds
2025-12-29 23:29:44,449 [INFO] openai._base_client - Retrying request to /chat/completions in 0.884530 seconds
2025-12-29 23:29:46,776 [INFO] openrouter_agent - run_agent_query finished.


Agent answer:
Hello! It sounds like you might be experiencing some technical difficulties. Have you tried turning your computer or modem off and then back on? This often resolves many common issues. Give that a try and let me know if you still need assistance!


{
    "name": "chat mistralai/devstral-2512:free",
    "context": {
        "trace_id": "0xc3b48d25b661b28c9275fed8737aab86",
        "span_id": "0xeac28e1262a9cf90",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xd2cb22c4b9274a07",
    "start_time": "2025-12-30T04:29:41.791356Z",
    "end_time": "2025-12-30T04:29:46.776052Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.operation.name": "chat",
        "gen_ai.system": "litellm",
        "gen_ai.request.model": "mistralai/devstral-2512:free",
        "server.address": "openrouter.ai",
        "model_request_parameters": "{\"function_tools\": [], \"builtin_tools\": [], \"output_mode\": \"text\", \"output_object\": null, \"output_tools\": [], \"prompted_output_template\": null, \"allow_text_output\": true, \"allow_image_output\": false}",
        "gen_ai.input.messages": "[{\"role\": \"system\", \"parts\": [{\"type\": \"text\", \"content\": \"You are a